`Auto3D` is mainly designed for generating low-energy 3D structures from the SMILES. It aslo provides a wrapper function `calc_thermo` to get enthalpy, entropy and Gibbs free energy from the 3D structures. Please note that the thermodynamic calculations are in vaccum.

In [1]:
import os, sys
root = os.path.dirname(os.path.dirname(os.path.abspath("__file__")))
sys.path.append(root)

import Auto3D
from Auto3D.auto3D import options, main

In [2]:
#Always ensure that you have the latest version
print(Auto3D.__version__)

0.1.9


# Generate low-energy conformers with Auto3D

Thermodynamic calculations requires proper 3D structures as the input. This time, we will use `ANI2x` as the optimizing engine. `ANI2x` and `ANI2xt` are easy to converge in the later thermodynamic analysis.

In [3]:
if __name__ == "__main__":
    path = os.path.join(root, "example/files/smiles.smi")
    args = options(path, k=1, optimizing_engine="ANI2x", use_gpu=False)   #args specify the parameters for Auto3D 
    out = main(args)            #main acceps the parameters and run Auto3D
    print(out)

Checking input file...
	There are 4 SMILES in the input file /Users/liu5/Documents/Auto3D_pkg/example/files/smiles.smi. 
	All SMILES and IDs are valid.
Suggestions for choosing isomer_engine and optimizing_engine: 
	Isomer engine options: RDKit and Omega.
	Optimizing engine options: ANI2x, ANI_2xt and AIMNET.
There are 4 SMILES, available memory is 16.0 GB.
The task will be divided into 1 jobs.
Job1, number of inputs: 4


Isomer generation for job1
Enumerating conformers/rotamers, removing duplicates...


Optimizing on job1


100%|██████████| 4/4 [00:00<00:00, 74.66it/s]
/opt/anaconda3/envs/py39/lib/python3.9/site-packages/torch/functional.py:1069: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at  /Users/distiller/project/conda/conda-bld/pytorch_1640812094853/work/aten/src/ATen/native/TensorShape.cpp:2157.)
  return _VF.cartesian_prod(tensors)  # type: ignore[attr-defined]


Preparing for parallel optimizing... (Max optimization steps: 5000)
Total 3D conformers: 14


  0%|          | 0/5000 [00:00<?, ?it/s]/opt/anaconda3/envs/py39/lib/python3.9/site-packages/torchani/aev.py:236: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  pair_sizes = counts * (counts - 1) // 2
 10%|▉         | 497/5000 [00:38<03:43, 20.12it/s]

Total 3D structures: 14  Converged: 7   Active: 7


...

Total 3D structures: 14  Converged: 13   Active: 1


 40%|███▉      | 1995/5000 [01:22<00:56, 52.84it/s]

Total 3D structures: 14  Converged: 13   Active: 1


 47%|████▋     | 2330/5000 [01:28<01:41, 26.28it/s]


All structures converged at step 2331:   Total 3D structures: 14  Converged: 14   Active: 0
Beggin to slelect structures that satisfy the requirements...
Energy unit: Hartree if implicit.
Program running time: 1 minutes
/Users/liu5/Documents/Auto3D_pkg/example/files/20220627-111133_smiles/smiles_out.sdf


# Calculate thermodynamic properties with the 3D structures

In [4]:
from Auto3D.ASE.thermo import calc_thermo

In [7]:
"""
You need to define a function that gets the unique ID and temperature for each molecules in the input path.
The mol object is an OpenBabel pybel Molecule object.

For example, the following functions works for Auto3D main output. It sets the thermodynamic calculation temperature at 298 K and molecular ID.
You can customize the function for other special needs.
"""
def get_mol_idx_t(mol):
    id = mol.data["ID"]
    t = 298
    return (id, t)


out_thermo = calc_thermo(out, "ANI2x", get_mol_idx_t, opt_tol=0.003)
print(out_thermo)  #enthalpy, entropy and Gibbs free energy are stored in the SDF file

      Step     Time          Energy         fmax
BFGS:    0 11:17:09   -13045.662981        0.0057
BFGS:    1 11:17:09   -13045.662983        0.0045
BFGS:    2 11:17:09   -13045.662985        0.0026


/opt/anaconda3/envs/py39/lib/python3.9/site-packages/torchani/aev.py:236: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  pair_sizes = counts * (counts - 1) // 2


Enthalpy components at T = 298.00 K:
E_pot             -13045.663 eV
E_ZPE                  5.113 eV
Cv_trans (0->T)        0.039 eV
Cv_rot (0->T)          0.039 eV
Cv_vib (0->T)          0.197 eV
(C_v -> C_p)           0.026 eV
-------------------------------
H                 -13040.250 eV
Entropy components at T = 298.00 K and P = 101325.0 Pa:
                           S               T*S
S_trans (1 bar)    0.0017751 eV/K        0.529 eV
S_rot              0.0013168 eV/K        0.392 eV
S_elec             0.0000000 eV/K        0.000 eV
S_vib              0.0012337 eV/K        0.368 eV
S (1 bar -> P)    -0.0000011 eV/K       -0.000 eV
-------------------------------------------------
S                  0.0043244 eV/K        1.289 eV
Enthalpy components at T = 298.00 K:
E_pot             -13045.663 eV
E_ZPE                  5.113 eV
Cv_trans (0->T)        0.039 eV
Cv_rot (0->T)          0.039 eV
Cv_vib (0->T)          0.197 eV
(C_v -> C_p)           0.026 eV
-------------------------

In [8]:
help(calc_thermo)

Help on function calc_thermo in module Auto3D.ASE.thermo:

calc_thermo(path: str, model_name: str, get_mol_idx_t: <built-in function callable>, gpu_idx=0, opt_tol=0.0005, opt_steps=5000)
    ASE interface for calculation thermo properties using ANI2xt and AIMNET
    path: Input sdf file
    model_name: ANI2x, ANI2xt or AIMNET
    get_mol_idx_t: a functioin that returns (idx, T) from a pybel mol object
    gpu_idx: GPU cuda index
    opt_tol: Convergence_threshold for geometry optimization
    opt_steps: Maximum geometry optimizaiton steps

